use python 3.9.13


In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# pip install -r ../requirements.txt

# Rancangan


---

[ Pilih Soal ]: ⬇️ Stimulus A / Stimulus B

[ Stimulus ]:
Pemanasan global terjadi karena...

[ Pertanyaan ]:
Apa yang menjadi tantangan...

[ Jawaban Siswa ]:
✎ **\*\*\*\***\_**\*\*\*\***--

[🔍 Nilai Jawaban]

📊 Hasil:
✅ Jawaban diklasifikasikan sebagai "Relevan dan Benar" (Label: 1)

🧠 Alasan: Jawaban mengandung kata kunci "pindah", "ekonomi", "sosial"


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torch.optim import AdamW  # Perubahan di sini
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, f1_score
import optuna
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer, BertModel

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

---


# Data Understanding


In [ ]:
train_a = pd.read_csv('../Dataset/Data_A/data_train_A.csv')
dev_a = pd.read_csv('../Dataset/Data_A/data_dev_A.csv')
test_a = pd.read_csv('../Dataset/Data_A/data_test_A.csv')
train_b = pd.read_csv('../Dataset/Data_B/data_train_B.csv')
dev_b = pd.read_csv('../Dataset/Data_B/data_dev_B.csv')
test_b = pd.read_csv('../Dataset/Data_B/data_test_B.csv')

In [ ]:
print("Data Train A:")
print(train_a.head())
print("\nKolom:", train_a.columns.tolist())

In [ ]:
test_a

In [ ]:
print("Data Train B:")
print(train_b.head())
print("\nKolom:", train_b.columns.tolist())

In [ ]:
stimulus_a = ["Pemanasan global terjadi karena peningkatan produksi karbon dioksida yang dihasilkan oleh pembakaran fosil dan konsumsi bahan bakar yang tinggi.",
"Salah satu akibat adalah mencairnya es abadi di kutub utara dan selatan yang menimbulkan naiknya ketinggian air laut.",
"kenaikan air laut akan terjadi terus menerus meskipun dalam hitungan centimeter akan mengakibatkan perubahan yang signifikan.",
"Film “Waterworld”, adalah film fiksi ilmiah yang menunjukkan akibat adanya pemanasan global yang sangat besar sehingga menyebabkan bumi menjadi tertutup oleh lautan.",
"Negara-negara dan daratan yang dulunya kering menjadi tengelamn karena terjadi kenaikan permukaan air laut.",
"Penduduk yang dulunya bisa berkehidupan bebas menjadi terpaksa mengungsi ke daratan yang lebih tinggi atau tinggal diatas air.",
"Apa yang akan menjadi tantangan bagi suatu penduduk ketika terjadi situasi daratan tidak dapat ditinggali kembali karena tengelam oleh naiknya air laut."]

stimulus_b = ["Sebuah toko baju berkonsep self-service menawarkan promosi dua buah baju bertema tahun baru seharga Rp50.000,00. sebelum baju bertema tahun baru dibagikan kepada pembeli, sebuah layar akan menampilkan tampilan gambar yang menampilkan kondisi kerja di dalam sebuah pabrik konveksi/pembuatan baju. ",
"Kemudian pembeli diberi program pilihan untuk menyelesaikan pembeliannya atau menyumpangkan Rp50.000,00 untuk dijadikan donasi pembagian baju musim dingin di suatu daerah yang membutuhkan.",
"Delapan dari sepuluh pembeli memilih untuk memberikan donasi.",
"Menurut anda mengapa banyak dari pembeli yang memilih berdonasi?"]

In [ ]:
stimulus_a_text = " ".join(stimulus_a)
stimulus_b_text = " ".join(stimulus_b)

for df in [train_a, dev_a, test_a]:
    df["TEXT"] = stimulus_a_text + " [SEP] " + df["RESPONSE"]

for df in [train_b, dev_b, test_b]:
    df["TEXT"] = stimulus_b_text + " [SEP] " + df["RESPONSE"]


In [ ]:
print(train_a.info())
print(train_a.describe(include='all'))  
print(train_b.info())
print(train_b.describe(include='all'))  

In [ ]:
sns.countplot(x='LABEL', data=train_a)
plt.title('Distribusi Label (Data_A Train)')
plt.show()

sns.countplot(x='LABEL', data=train_b)
plt.title('Distribusi Label (Data_B Train)')
plt.show()

---


# Preprocess


In [ ]:
stopwords_ukara = {'yang', 'lebih', 'untuk', 'akan', 'mereka', 'dan'}

In [ ]:
def preprocess(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords_ukara]
    return " ".join(tokens)


In [ ]:
train_a["clean_text"] = train_a["TEXT"].apply(preprocess)
train_b["clean_text"] = train_b["TEXT"].apply(preprocess)
test_a["clean_text"] = test_a["TEXT"].apply(preprocess)
test_b["clean_text"] = test_b["TEXT"].apply(preprocess)
dev_a["clean_text"] = dev_a["TEXT"].apply(preprocess)
dev_b["clean_text"] = dev_b["TEXT"].apply(preprocess)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')


In [ ]:
def encode_data(texts, labels, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    
    return input_ids, attention_masks, labels

---


# Model


In [ ]:
class IndoBERTClassifier(torch.nn.Module):
    def __init__(self, num_labels=2):
        super(IndoBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('indobenchmark/indobert-base-p1')
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, num_labels)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
def train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, device, epochs=4):
    best_val_f1 = 0
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for batch in train_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            
            model.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = torch.nn.CrossEntropyLoss()(outputs, labels)
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        
        avg_train_loss = total_loss / len(train_dataloader)
        
        # Validation
        val_f1, val_accuracy = evaluate_model(model, val_dataloader, device)
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train loss: {avg_train_loss:.4f}")
        print(f"Validation F1: {val_f1:.4f}, Accuracy: {val_accuracy:.4f}")
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), 'best_model.pt')
    
    return model

In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask)
        
        logits = outputs.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        
        predictions.extend(np.argmax(logits, axis=1).flatten())
        true_labels.extend(label_ids.flatten())
    
    f1 = f1_score(true_labels, predictions)
    accuracy = accuracy_score(true_labels, predictions)
    return f1, accuracy


In [ ]:
def run_experiment(train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, params):
    # Encode the data
    train_input_ids, train_attention_masks, train_labels = encode_data(train_texts, train_labels, tokenizer)
    val_input_ids, val_attention_masks, val_labels = encode_data(val_texts, val_labels, tokenizer)
    test_input_ids, test_attention_masks, test_labels = encode_data(test_texts, test_labels, tokenizer)
    
    # Create DataLoaders
    train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=params['batch_size'])
    
    val_data = TensorDataset(val_input_ids, val_attention_masks, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=params['batch_size'])
    
    test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=params['batch_size'])
    
    # Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = IndoBERTClassifier(num_labels=2)
    model.to(device)
    
    # Optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=params['learning_rate'], eps=1e-8)
    total_steps = len(train_dataloader) * params['epochs']
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=0,
                                              num_training_steps=total_steps)
    
    # Train the model
    model = train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, device, params['epochs'])
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pt'))
    
    # Evaluate on test set
    test_f1, test_accuracy = evaluate_model(model, test_dataloader, device)
    
    print(f"\nTest Results - Accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}")
    
    return {
        'test_accuracy': test_accuracy,
        'test_f1': test_f1
    }


In [ ]:
def objective(trial, train_texts, train_labels, val_texts, val_labels):
    params = {
        'batch_size': trial.suggest_categorical('batch_size', [8, 16, 32]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True),
        'epochs': trial.suggest_int('epochs', 3, 5)
    }
    
    # Run training with current hyperparameters
    results = run_experiment(
        train_texts, train_labels, 
        val_texts, val_labels,
        val_texts, val_labels,  # Using val as test for optimization
        params
    )
    
    return results['test_f1']


---


## Data A


In [ ]:
train_texts_a = train_a['clean_text'].tolist()
train_labels_a = train_a['LABEL'].values
val_texts_a = dev_a['clean_text'].tolist()
val_labels_a = dev_a['LABEL'].values
test_texts_a = test_a['clean_text'].tolist()
test_labels_a = test_a['LABEL'].values

In [ ]:
study_a = optuna.create_study(direction='maximize')
study_a.optimize(lambda trial: objective(trial, train_texts_a, train_labels_a, val_texts_a, val_labels_a), n_trials=10)
best_params_a = study_a.best_params
print("Best hyperparameters for Dataset A:", best_params_a)

In [ ]:
final_results_a = run_experiment(
    train_texts_a, train_labels_a,
    val_texts_a, val_labels_a,
    test_texts_a, test_labels_a,
    best_params_a
)

---


## Data B


In [ ]:
train_texts_b = train_b['clean_text'].tolist()
train_labels_b = train_b['LABEL'].values
val_texts_b = dev_b['clean_text'].tolist()
val_labels_b = dev_b['LABEL'].values
test_texts_b = test_b['clean_text'].tolist()
test_labels_b = test_b['LABEL'].values

In [ ]:

study_b = optuna.create_study(direction='maximize')
study_b.optimize(lambda trial: objective(trial, train_texts_b, train_labels_b, val_texts_b, val_labels_b), n_trials=10)
best_params_b = study_b.best_params
print("Best hyperparameters for Dataset B:", best_params_b)

In [ ]:
final_results_b = run_experiment(
    train_texts_b, train_labels_b,
    val_texts_b, val_labels_b,
    test_texts_b, test_labels_b,
    best_params_b
)

In [ ]:
print("\nFinal Results:")
print(f"Dataset A - Test Accuracy: {final_results_a['test_accuracy']:.4f}, F1: {final_results_a['test_f1']:.4f}")
print(f"Dataset B - Test Accuracy: {final_results_b['test_accuracy']:.4f}, F1: {final_results_b['test_f1']:.4f}")

# Opsi

1. Bert prepocessing + Model
2. Bert Prepocessing, Model Neural
3. Bert ...
